In [ ]:
# 필요 패키지 설치 (터미널 또는 노트북 셀에서 한 번만 실행)
# pip install langchain openai python-dotenv

from enum import Enum
from langchain.output_parsers import EnumOutputParser
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException
from enum import Enum

# 감정 Enum 정의
class MovieSentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "보통"

# Enum 파서 생성
enum_parser = EnumOutputParser(enum=MovieSentiment)
format_instructions = enum_parser.get_format_instructions()

# 프롬프트 템플릿 정의
template = """
당신은 영화 리뷰 감정 분석 전문가입니다.
아래 리뷰의 감정을 분석하고 반드시 "긍정", "부정", "보통" 중 하나만 출력하세요.

리뷰: "{review}"

{format_instructions}

중요 규칙:
1. 반드시 "긍정", "부정", "보통" 중 하나의 단어만 출력하세요
2. 다른 설명이나 부가 설명은 하지 마세요
3. 오직 하나의 단어만 출력하세요

답변:"""

prompt = ChatPromptTemplate.from_template(template).partial(format_instructions=format_instructions)

# 모델 설정 (Groq API나 OpenAI API 등 사용자에 맞게)
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

# OutputFixingParser는 선택사항
fixing_parser = OutputFixingParser.from_llm(parser=enum_parser, llm=model)

# 분석 함수 정의
def safe_movie_sentiment(review, use_fixing_parser=True):
    try:
        chain = prompt | model | (fixing_parser if use_fixing_parser else enum_parser)
        result = chain.invoke({"review": review})
        return result, None
    except OutputParserException as e:
        return None, f"파싱 오류: {str(e)}"
    except Exception as e:
        return None, f"일반 오류: {str(e)}"

# 테스트 리뷰 리스트
test_reviews = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.",
    "OST는 좋았지만 전반적으로 지루했어요.",
    "연출, 연기, 스토리 모두 완벽했습니다!"
]

# 실행
def run_movie_sentiment_analysis():
    print("=== 영화 리뷰 감정 분석 결과 ===")
    success, total = 0, len(test_reviews)

    for i, review in enumerate(test_reviews, 1):
        print(f"\n{i}. 리뷰: {review}")
        result, error = safe_movie_sentiment(review)
        if result:
            print(f"   감정: {result.value}")
            success += 1
        else:
            print(f"   오류 발생: {error}")
            print("   기본 파서로 재시도 중...")
            result2, error2 = safe_movie_sentiment(review, use_fixing_parser=False)
            if result2:
                print(f"   감정: {result2.value} (기본 파서 성공)")
                success += 1
            else:
                print(f"   재시도 실패: {error2}")

    print(f"\n[요약] 성공 {success}/{total} ({success / total * 100:.1f}%)")

# 실행 (API 설정 필요)
try:
    run_movie_sentiment_analysis()
except Exception as e:
    print("\n⚠️ 실행 실패: API 키 누락 또는 네트워크 문제")
    print(f"오류: {e}")

=== 영화 리뷰 감정 분석 결과 ===

1. 리뷰: 이 영화 정말 재미없어요. 시간 낭비였습니다.
   감정: 부정

2. 리뷰: 배우들의 연기가 훌륭하고 스토리도 감동적이었어요!
   감정: 긍정

3. 리뷰: 그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.
   감정: 보통

4. 리뷰: OST는 좋았지만 전반적으로 지루했어요.
   감정: 부정

5. 리뷰: 연출, 연기, 스토리 모두 완벽했습니다!
   감정: 긍정

[요약] 성공 5/5 (100.0%)
